In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Agregar la carpeta raíz del proyecto al path
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import pandas as pd
print("Cargando datos procesados...")
train_df = pd.read_csv('../temp/train_processed.csv')
test_df = pd.read_csv('../temp/test_processed.csv')

# 2. Separar X e y
X_train = train_df.drop(columns=['popularity'])
y_train = train_df['popularity']

X_test = test_df.drop(columns=['popularity'])
y_test = test_df['popularity']

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

Cargando datos procesados...
X_train: (333345, 43)
X_test: (83336, 43)


In [3]:
from helpers.trainer import ModelTrainer
trainer = ModelTrainer(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    target_col='popularity'
)

✓ ModelTrainer inicializado - Target: popularity
  Train: 333,345 muestras, 43 features
  Test: 83,336 muestras
  Target range: [0.0, 96.0]
  Target mean: 29.3 ± 15.8


In [4]:
custom_grid = {
    'n_estimators': [200, 300],
    'learning_rate': [0.05, 0.1],
    'max_depth': [5, 7]
} # Pueden configurar esta grid como deseen o quitarla para usar la por defecto

In [5]:
trainer.train_model(
    model_name='catboost',
    cv=5,
    param_grid=None, # Pueden pasar custom_grid o None para usar la grid por defecto
)


ENTRENANDO: CATBOOST
Grid de búsqueda:
  iterations: [100, 200, 300]
  learning_rate: [0.01, 0.05, 0.1]
  depth: [4, 6, 8]
  l2_leaf_reg: [1, 3, 5]

 Buscando mejores hiperparámetros con 5-Fold CV...
  Total combinaciones: 81

  [1/81] Probando: {'iterations': 100, 'learning_rate': 0.01, 'depth': 4, 'l2_leaf_reg': 1}
    MSE CV: 211.7954 (±1.4048)
    ✓ Nuevo mejor score: 211.7954

  [2/81] Probando: {'iterations': 100, 'learning_rate': 0.01, 'depth': 4, 'l2_leaf_reg': 3}
    MSE CV: 211.7981 (±1.4067)

  [3/81] Probando: {'iterations': 100, 'learning_rate': 0.01, 'depth': 4, 'l2_leaf_reg': 5}
    MSE CV: 211.8182 (±1.4026)

  [4/81] Probando: {'iterations': 100, 'learning_rate': 0.01, 'depth': 6, 'l2_leaf_reg': 1}
    MSE CV: 204.4420 (±1.3484)
    ✓ Nuevo mejor score: 204.4420

  [5/81] Probando: {'iterations': 100, 'learning_rate': 0.01, 'depth': 6, 'l2_leaf_reg': 3}
    MSE CV: 204.4454 (±1.3504)

  [6/81] Probando: {'iterations': 100, 'learning_rate': 0.01, 'depth': 6, 'l2_leaf_r

{'model_name': 'catboost',
 'best_params': {'iterations': 300,
  'learning_rate': 0.1,
  'depth': 8,
  'l2_leaf_reg': 1},
 'cv_score': np.float64(151.8566224535237),
 'train_metrics': {'mse': 147.376556441417,
  'rmse': np.float64(12.139874646857644),
  'mae': 9.617504172596718,
  'r2': 0.4069133308509165},
 'test_metrics': {'mse': 153.06737871156744,
  'rmse': np.float64(12.372040200046532),
  'mae': 9.771130320564847,
  'r2': 0.3861897362237361},
 'training_time': 2195.819221,
 'model': <catboost.core.CatBoostRegressor at 0x77f95cd407d0>}

In [6]:
print(trainer.results.keys())  #Debe incluir el modelo entrenado

dict_keys(['catboost'])


In [8]:
results = trainer.results['catboost'] #No da error si el modelo fue entrenado correctamente
print(f"XGBoost CV MSE: {results['cv_score']:.4f}")
print(f"XGBoost Test R²: {results['test_metrics']['r2']:.4f}")

XGBoost CV MSE: 151.8566
XGBoost Test R²: 0.3862


In [ ]:
trainer.save_model(model_name='catboost', path='../models/catboost_model.joblib')

✓ Modelo guardado: ../models/catboost_model.joblib
✓ Metadata guardada: ../models/catboost_model_metadata.joblib


In [11]:

from helpers.trainer import load_model
# Para cargar el modelo guardado posteriormente
loaded_model,metadata = load_model(path='../models/catboost_model.joblib')

loaded_model

✓ Modelo: catboost
  CV MSE: 151.8566
  Test R²: 0.3862
  Test RMSE: 12.3720
